### 사용할 라이브러리

In [ ]:
### 기본 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 형태소 분석
from konlpy.tag import Okt

### 단어 분할
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### 정규표현식(Regular Expression; regex) 
import re

### 데이터셋 읽어들이기

In [2]:
### ratins_train and ratings_test 파일의 데이터셋 읽어들이기
# - 변수명 : train_data, test_data
# - read_tabel() 함수 사용
train_data = pd.read_table("./files/ratings_train.txt")
test_data = pd.read_table("./files/ratings_test.txt")

In [3]:
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


### 종속변수로 사용할 label의 고유한 범주 확인하기

In [4]:
### label 데이터의 범주별 갯수 확인하기
print(train_data["label"].value_counts())
print(test_data["label"].value_counts())

### 훈련 및 테스트 데이터의 범주는 0과 1로,
#   - 범주별 데이터 건수는 고르게 분포되어 있음

label
0    75173
1    74827
Name: count, dtype: int64
label
1    25173
0    24827
Name: count, dtype: int64


### 독립변수로 사용할 document 데이터 전처리하기

In [5]:
train_data["document"]

0                                       아 더빙.. 진짜 짜증나네요 목소리
1                         흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                             교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                  인간이 문제지.. 소는 뭔죄인가..
149996                                        평점이 너무 낮아서...
149997                      이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998                          청춘 영화의 최고봉.방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 150000, dtype: object

In [6]:
### 정규표현식을 이용하여 영어/숫자/특수문자 모두 제거하기
# - 한글과 공백을 제외한 모든것 제거하기
# - 정규표현식 : 문장 내에 특정 문자의 패턴을 이용하여 필터링하는 기법
# - 사용함수 : replace("정규표현식", "바꿀값", regex=True)
#            : regex=True -> 정규표현식을 이용하여 필터링 하겠다는 정의
# [^ㄱ-ㅎㅏ-ㅣ가-힣 ] : 한글과 공백이 아닌(^) 모든[] 것을 의미함
train_data["document"] = train_data["document"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
test_data["document"]  = test_data["document"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)


In [7]:
train_data.head(1)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0


In [8]:
test_data.head(1)

,id,document,label
0,6270596,굳 ㅋ,1


### 독립변수(document) 내에 공백(white space)만 있는 경우 제거하기

In [9]:
### 데이터 내에 공백(white space)은 문자로 인식하기 때문에 
# - 공백을 ""로 치환하면 Nan이 됨
# - Nan 처리 후 기존의 결측치 제거 방식을 사용함

### 공백을 ""로 치환하기 : 정규표현식 사용
# - ^ + : 공백 뒤에 문자로 시작(+)되지 않는(^) 것은 제거
train_data["document"] = train_data["document"].str.replace("^ +", "", regex=True)

### ""로 변환된 값들을 다시 nan으로 치환
train_data["document"].replace("", np.nan, inplace=True)

### 기존의 결측치 처리방식으로 nan에 대해서 모든 행 삭제하기
# - how="any" : 행 또는 열 중에 하나라도 NaN이 있으면 해당 행 삭제
train_data = train_data.dropna(how="any")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7896\3775835994.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["document"].replace("", np.nan, inplace=True)


In [10]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 148740 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        148740 non-null  int64 
 1   document  148740 non-null  object
 2   label     148740 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.5+ MB


In [11]:
### 데이터 내에 공백(white space)은 문자로 인식하기 때문에 
# - 공백을 ""로 치환하면 Nan이 됨
# - Nan 처리 후 기존의 결측치 제거 방식을 사용함

### 공백을 ""로 치환하기 : 정규표현식 사용
# - ^ + : 공백 뒤에 문자로 시작(+)되지 않는(^) 것은 제거
test_data["document"] = test_data["document"].str.replace("^ +", "", regex=True)

### ""로 변환된 값들을 다시 nan으로 치환
test_data["document"].replace("", np.nan, inplace=True)

### 기존의 결측치 처리방식으로 nan에 대해서 모든 행 삭제하기
# - how="any" : 행 또는 열 중에 하나라도 NaN이 있으면 해당 행 삭제
test_data = test_data.dropna(how="any")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7896\4167284106.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_data["document"].replace("", np.nan, inplace=True)


In [12]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49575 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49575 non-null  int64 
 1   document  49575 non-null  object
 2   label     49575 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


### 형태소 분석하기

In [13]:
### 불용어 정의하기
stopwords = ["아", "의", "가", "이", "은", "들", "는", "좀", 
             "잘", "걍", "과", "도", "를", "으로", "자",
             "에", "와", "한", "하다", "흠", "ㅠㅠ", "ㅋ", 
             "ㅋㅋ", "ㅎ", "ㅎㅎ", "전"]

In [14]:
### 형태소 분석기 생성하기
okt = Okt()
okt

In [15]:
print(train_data["document"][0])
okt.morphs(train_data["document"][0])

아 더빙 진짜 짜증나네요 목소리


['아', '더빙', '진짜', '짜증나네요', '목소리']

In [16]:
### 형태소 분할된 후 불용어 처리 후 각 행의 단어들을 리스트 변수에 담기
# - document의 모든 행 내에 단어들을 행단위로 단어 분리 후 리스트 변수에 담기
# - 변수명 : X_train

X_train = []

for sentence in train_data["document"] :
    # 형태소 분석
    tokenized_sentence = okt.morphs(sentence)
    # 불용어 제거
    stopwords_sentence = [word for word in tokenized_sentence 
                                    if word not in stopwords]
    # 리스트에 담기
    X_train.append(stopwords_sentence)

In [17]:
### 훈련 데이터 : 2차원
X_train

[['더빙', '진짜', '짜증나네요', '목소리'],
 ['포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍지', '않구나'],
 ['너', '무재', '밓었', '다그', '래서', '보는것을', '추천', '다'],
 ['교도소', '이야기', '구먼', '솔직히', '재미', '없다', '평점', '조정'],
 ['사이',
  '몬페',
  '그',
  '익살스런',
  '연기',
  '돋보였던',
  '영화',
  '스파이더맨',
  '에서',
  '늙어',
  '보이기만',
  '했던',
  '커스틴',
  '던스트',
  '너무나도',
  '이뻐',
  '보였다'],
 ['막',
  '걸음',
  '마',
  '뗀',
  '세',
  '부터',
  '초등학교',
  '학년',
  '생인',
  '살용',
  '영화',
  'ㅋㅋㅋ',
  '별',
  '반개',
  '아까',
  '움'],
 ['원작', '긴장감', '을', '제대로', '살려내지못', '했다'],
 ['별',
  '반개',
  '아깝다',
  '욕',
  '나온다',
  '이응경',
  '길용우',
  '연',
  '기',
  '생활',
  '몇',
  '년',
  '인지',
  '정말',
  '발',
  '로',
  '해도',
  '그것',
  '보단',
  '낫겟다',
  '납치',
  '감금',
  '만',
  '반복',
  '반복',
  '드라마',
  '가족',
  '없다',
  '연기',
  '못',
  '하는',
  '사람',
  '만',
  '모',
  '엿',
  '네'],
 ['액션', '없는데도', '재미', '있는', '몇', '안되는', '영화'],
 ['왜케',
  '평점',
  '낮은건데',
  '꽤',
  '볼',
  '만',
  '데',
  '헐리우드',
  '식',
  '화려함에만',
  '너무',
  '길들여져',
  '있나'],
 ['인피니트', '짱', '이다', '진짜', '짱', '이다'],
 ['볼

In [18]:
### 형태소 분할된 후 불용어 처리 후 각 행의 단어들을 리스트 변수에 담기
# - document의 모든 행 내에 단어들을 행단위로 단어 분리 후 리스트 변수에 담기
# - 변수명 : X_test

X_test = []

for sentence in test_data["document"] :
    # 형태소 분석
    tokenized_sentence = okt.morphs(sentence)
    # 불용어 제거
    stopwords_sentence = [word for word in tokenized_sentence 
                                    if word not in stopwords]
    # 리스트에 담기
    X_test.append(stopwords_sentence)

In [19]:
print(len(X_train), len(X_test))

148740 49575


### 각 단어들에 번호 부여하기

In [20]:
### 각 단어들에 번호 부여하기 : Tokenizer() 사용
tokenizer = Tokenizer()
# - 2차원 리스트 내에 모든 단어들을 추출하여 번호를 부여시킴
#  -> 반환 결과는 딕셔너리 타입
tokenizer.fit_on_texts(X_train)
tokenizer.word_index

{'영화': 1,
 '을': 2,
 '너무': 3,
 '다': 4,
 '정말': 5,
 '적': 6,
 '만': 7,
 '진짜': 8,
 '로': 9,
 '점': 10,
 '에서': 11,
 '연기': 12,
 '평점': 13,
 '것': 14,
 '최고': 15,
 '내': 16,
 '그': 17,
 '나': 18,
 '안': 19,
 '인': 20,
 '이런': 21,
 '스토리': 22,
 '생각': 23,
 '못': 24,
 '왜': 25,
 '드라마': 26,
 '게': 27,
 '이다': 28,
 '감동': 29,
 '사람': 30,
 '보고': 31,
 '하는': 32,
 '하고': 33,
 '말': 34,
 '고': 35,
 '더': 36,
 '배우': 37,
 '때': 38,
 '감독': 39,
 '거': 40,
 '그냥': 41,
 '요': 42,
 '본': 43,
 '재미': 44,
 '시간': 45,
 '내용': 46,
 '뭐': 47,
 '까지': 48,
 '중': 49,
 '쓰레기': 50,
 '보다': 51,
 '없는': 52,
 '네': 53,
 '수': 54,
 '지': 55,
 '봤는데': 56,
 '작품': 57,
 '사랑': 58,
 '할': 59,
 '없다': 60,
 '볼': 61,
 '하나': 62,
 '다시': 63,
 '마지막': 64,
 '좋은': 65,
 '이건': 66,
 '정도': 67,
 '저': 68,
 '같은': 69,
 '완전': 70,
 '입니다': 71,
 '있는': 72,
 'ㅋㅋㅋ': 73,
 '처음': 74,
 '장면': 75,
 '액션': 76,
 '주인공': 77,
 '걸': 78,
 '이렇게': 79,
 '보는': 80,
 '최악': 81,
 '개': 82,
 '하': 83,
 '돈': 84,
 '이야기': 85,
 '지금': 86,
 '별로': 87,
 '봐도': 88,
 '느낌': 89,
 '임': 90,
 '참': 91,
 'ㅡㅡ': 92,
 '연출': 93,
 '라': 94,
 '

In [21]:
len(tokenizer.word_index)

99996

In [22]:
### 단어와 빈도(번호)를 key, value로 조회시켜줌(items())
tokenizer.word_index.items()

dict_items([('영화', 1), ('을', 2), ('너무', 3), ('다', 4), ('정말', 5), ('적', 6), ('만', 7), ('진짜', 8), ('로', 9), ('점', 10), ('에서', 11), ('연기', 12), ('평점', 13), ('것', 14), ('최고', 15), ('내', 16), ('그', 17), ('나', 18), ('안', 19), ('인', 20), ('이런', 21), ('스토리', 22), ('생각', 23), ('못', 24), ('왜', 25), ('드라마', 26), ('게', 27), ('이다', 28), ('감동', 29), ('사람', 30), ('보고', 31), ('하는', 32), ('하고', 33), ('말', 34), ('고', 35), ('더', 36), ('배우', 37), ('때', 38), ('감독', 39), ('거', 40), ('그냥', 41), ('요', 42), ('본', 43), ('재미', 44), ('시간', 45), ('내용', 46), ('뭐', 47), ('까지', 48), ('중', 49), ('쓰레기', 50), ('보다', 51), ('없는', 52), ('네', 53), ('수', 54), ('지', 55), ('봤는데', 56), ('작품', 57), ('사랑', 58), ('할', 59), ('없다', 60), ('볼', 61), ('하나', 62), ('다시', 63), ('마지막', 64), ('좋은', 65), ('이건', 66), ('정도', 67), ('저', 68), ('같은', 69), ('완전', 70), ('입니다', 71), ('있는', 72), ('ㅋㅋㅋ', 73), ('처음', 74), ('장면', 75), ('액션', 76), ('주인공', 77), ('걸', 78), ('이렇게', 79), ('보는', 80), ('최악', 81), ('개', 82), ('하', 83), ('돈', 84), ('이야기', 85), (

In [23]:
### 말뭉치 갯수 정의하기
# - 빈도가 작은 단어의 갯수는 제외시키기

### 사용빈도가 작은 단어의 수 확인하기

### 임시로 기준 단어의 갯수 정의하기(비교 기준으로 사용)
# - 기준 갯수보다 작은 단어 빈도 확인을 위해
threshold = 3

# 총 단어의 수 
total_cnt = len(tokenizer.word_index)
print(total_cnt)
# 기준 단어의 갯수보다 작은 단어의 갯수 count(실제 확인할 값)
rare_cnt = 0

# 훈련 데이터에 사용할 전체 단어 빈도수 총합
total_freq = 0
# 기준 단어의 갯수보다 작은 단어의 등장 빈도수 총합
rare_freq = 0

### tokenizer의 word_counts를 items()를 사용하여 반복하기
for key, value in tokenizer.word_counts.items() :
    # 훈련 데이터의 전체 단어 빈도수 총합 누적하기
    total_freq = total_freq + value
    
    # 기준 단어의 갯수보다 작은 단어의 갯수 카운트 하기
    if value < threshold :
        # 작은 단어의 갯수 카운트
        rare_cnt = rare_cnt + 1
        
        # 작은 단어의 빈도수 총합 누적하기
        rare_freq = rare_freq + value

99996


In [24]:
### 출력하기
print(f"단어 집합의 전체 크기 : {total_cnt}")
print(f"등장 빈도가 {threshold - 1}개 이하인 단어의 수 : {rare_cnt}")
print(f"기준 갯수보다 작은 단어의 비율 : {(rare_cnt/total_cnt) * 100}%")
print(f"전체 단어 대비 기준보다 작은 단어 비율 : {(rare_freq/total_freq)/100}%")

단어 집합의 전체 크기 : 99996
등장 빈도가 2개 이하인 단어의 수 : 67672
기준 갯수보다 작은 단어의 비율 : 67.67470698827952%
전체 단어 대비 기준보다 작은 단어 비율 : 0.000498340736297111%


In [25]:
### 사용할 말뭉치 갯수 정의하기
# - 1을 더하는 이유 : 말뭉치에 없는 단어 처리를 위한 공간으로 사용(0번 인덱스 처리)
#                   : 패팅시에 빈 공간의 0번 인덱스에 대한 처리를 위함
vocab_size = total_cnt - rare_cnt + 1
print(vocab_size)

32325


### 텍스트를 숫자로 변환하기

In [26]:
### 말뭉치 사전을 기준으로 텍스트를 숫자로 변환하기
# 말뭉치 사전 생성하기 : vocab_size로 정의된 갯수에 대한 말뭉치 생성하기 위해
#  - Tokenizer 정의시 생성할 말뭉치 갯수 정의
tokenizer = Tokenizer(vocab_size)

### 단어를 정수로 변환하기 위한 패턴 찾기
#   (말뭉치 생성됨)
tokenizer.fit_on_texts(X_train)

In [27]:
### 찾은 패턴으로 훈련 및 테스트 데이터 변환하기
# - 말뭉치의 인덱스 번호로 텍스트를 매핑하여 변환
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [28]:
len(X_train), len(X_test)

(148740, 49575)

### 종속변수 numpy의 배열로 처리하기

In [29]:
### 종속변수 컬럼명 : label
y_train = np.array(train_data["label"])
y_test  = np.array(test_data["label"])

In [30]:
len(X_train), y_train.shape, len(X_test), y_test.shape

(148740, (148740,), 49575, (49575,))

### 말뭉치에 존재하지 않는 단어들의 번호는 비어 있기에 해단 부분 제거하기

In [31]:
### 비어 있는 부분에 대한 시퀀스의 위치 확인하기
# - enumerate : 데이터의 인덱스 번호와 해당 값을 동시에 추출
drop_train = [index for index, sentence in enumerate(X_train)
                        if len(sentence) < 1]
len(drop_train)

583

In [32]:
### X_train 데이터에서 drop_train의 인덱스 위치를 제외하고 추출하기
# - 변수명 : X_train에 제외한 결과 담기
X_train = [sentence for index, sentence in enumerate(X_train)
                      if index not in drop_train]
len(X_train)

148157

In [33]:
# X_train

In [34]:
### y_train 종속변수에서도 제거할 인덱스 위치의 행 제거하기
# - 변수명 : y_train
# - np.delete : 넘파이 배열 내에 특정 인덱스 영역을 행단위(axis=0)로 삭제하는 함수
y_train = np.delete(y_train, drop_train, axis=0)
len(X_train), y_train.shape

(148157, (148157,))

In [35]:
y_train

array([0, 1, 0, ..., 0, 1, 0], dtype=int64)

### 단어의 최대길이 및 평균길이 확인하기

In [36]:
### 단어의 최대길이
max_len = max(len(review) for review in X_train)

### 단어의 평균길이
mean_len = sum(map(len, X_train)) / len(X_train)

max_len, mean_len

(68, 10.391348366935075)

In [37]:
### max_len 정의하기
# - 평균 이상값을 사용하면 보편적 데이터들을 대부분 포함할 것으로 여겨짐
max_len = 11

In [38]:
### max_len 이내에 포함될 단어들의 비율 확인하기
# - max_len 이내에 비율이 너무 낮은 경우 max_len 수정이 필요하기 때문에
# - X_train 각 행의 단어의 갯수가 max_len 이내인 비율이 어느정도 인지 확인하기
max_len = 11
count = 0

for sentence in X_train :
    if len(sentence) <= max_len :
        count = count + 1
        
rate = count / len(X_train) * 100
print(f"max_len이하인 비율은 {rate}%")

### max_len 조정 방법
# - 100% 시점의 max_len 값을 찾으면 됨

max_len이하인 비율은 70.53531051519671%


### 데이터 스케일링 처리하기

In [39]:
### 데이터 스케일링 : max_len을 기준으로 모든 행의 데이터 갯수 맞추기
# X_train과 X_test 모두 padding 처리하기 : 변수명 동일하게 사용
X_train = pad_sequences(X_train, maxlen=max_len, padding="post", truncating="post")
X_test = pad_sequences(X_test, maxlen=max_len, padding="post", truncating="post")

In [40]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((148157, 11), (148157,), (49575, 11), (49575,))